<a href="https://colab.research.google.com/github/Krzesimir13/DataQuest/blob/main/laptops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>